In [2]:
# import libraries to get the machine ready

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import f1_score
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [10]:
# Dataset preparation

#df_perf = pandas.read_csv('training.csv', encoding='utf-8')
#df_non_perf = pandas.read_csv('testing.csv', encoding='utf-8')
#def tag(c):
#  if 'yes' in c.label:
#    return 1
#  else:
 #   return 0

#df_perf['tag'] = df_perf.apply(tag, axis=1)
#df_non_perf['tag'] = df_non_perf.apply(tag, axis=1)


# split the dataset into training and validation datasets
#df_training = pandas.DataFrame()
#df_training['label'] = df_perf['tag']
#df_training['text'] = df_perf['text']

#df = pandas.DataFrame()
#df['label'] = df_non_perf['tag']
#df['text'] = df_non_perf['text']

#df_1 = df.loc[df.label == 1]
#df_0 = df.loc[df.label == 0]

#print(len(df_1))
#print(len(df_0))

In [11]:
# split the dataset into training and validation datasets 
# train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

#train_x = df_training['text']
#valid_x = df['text']
#train_y = df_training['label']
#valid_y = df['label']

#print(len(valid_y))
#print(len(train_x))

In [3]:
#df1 = pandas.read_csv('testing_non_perf_vectorized.csv', encoding='utf-8')
df = pandas.read_csv('firefox-vector.csv', encoding='utf-8')
#df2 = pandas.read_csv('dataset-3-issues-tuned-vectorized.csv', encoding='utf-8')
#df = df1.append(df2)

def tag(c):
  if c.label == 'yes':
    return 1
  else:
    return 0

df['tag'] = df.apply(tag, axis=1)

In [4]:
from sklearn.model_selection import train_test_split
x = df['text']
y = df['tag']

train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.3)
print(train_x.shape, train_y.shape)
print(valid_x.shape, valid_y.shape)

(7000,) (7000,)
(3000,) (3000,)


In [5]:
print(train_y)

9298    0
7087    0
3476    0
9202    0
4657    0
       ..
44      0
6832    1
7535    0
9259    0
4167    0
Name: tag, Length: 7000, dtype: int64


In [6]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)


In [7]:
# 2. Feature Engineering
# 2.1 Count Vectors as features

# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(df['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [8]:

# 2.2 TF-IDF Vectors as features
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)


# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(df['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)


# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(df['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

In [9]:
# 2.3 Word Embeddings

# load the pre-trained word-embedding vectors 

embeddings_index = {}
for i, line in enumerate(open('data/wiki-news-300d-1M.vec',  encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(df['text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [10]:
#3. Model Building
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    #print(predictions)
    if is_neural_net:
        predictions = numpy.around(predictions, decimals=0)
    
    
    #print(valid_y)
    #print(predictions)
    print(classification_report(valid_y, predictions))
    print(confusion_matrix(valid_y, predictions))
    return metrics.accuracy_score(predictions, valid_y)



In [11]:
# 3.1 Naive Bayes

# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print( "NB, Count Vectors: ", accuracy)

             precision    recall  f1-score   support

          0       0.94      0.97      0.96      2728
          1       0.58      0.35      0.43       272

avg / total       0.90      0.92      0.91      3000

[[2658   70]
 [ 177   95]]
NB, Count Vectors:  0.9176666666666666


In [12]:
# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print("NB, WordLevel TF-IDF: ", accuracy)

             precision    recall  f1-score   support

          0       0.91      1.00      0.95      2728
          1       1.00      0.05      0.09       272

avg / total       0.92      0.91      0.88      3000

[[2728    0]
 [ 259   13]]
NB, WordLevel TF-IDF:  0.9136666666666666


In [13]:
# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print( "NB, N-Gram Vectors: ", accuracy)

             precision    recall  f1-score   support

          0       0.92      1.00      0.96      2728
          1       0.88      0.10      0.18       272

avg / total       0.91      0.92      0.89      3000

[[2724    4]
 [ 244   28]]
NB, N-Gram Vectors:  0.9173333333333333


In [14]:
# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print( "NB, CharLevel Vectors: ", accuracy)

             precision    recall  f1-score   support

          0       0.92      0.99      0.95      2728
          1       0.59      0.08      0.14       272

avg / total       0.89      0.91      0.88      3000

[[2713   15]
 [ 250   22]]
NB, CharLevel Vectors:  0.9116666666666666


In [15]:
# 3.2 Linear Classifier
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print( "LR, Count Vectors: ", accuracy)



             precision    recall  f1-score   support

          0       0.95      0.99      0.97      2728
          1       0.90      0.49      0.64       272

avg / total       0.95      0.95      0.94      3000

[[2713   15]
 [ 138  134]]
LR, Count Vectors:  0.949


In [16]:
## Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print( "LR, WordLevel TF-IDF: ", accuracy)

             precision    recall  f1-score   support

          0       0.92      1.00      0.96      2728
          1       0.92      0.18      0.30       272

avg / total       0.92      0.92      0.90      3000

[[2724    4]
 [ 224   48]]
LR, WordLevel TF-IDF:  0.924


In [17]:
# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print( "LR, N-Gram Vectors: ", accuracy)

             precision    recall  f1-score   support

          0       0.92      1.00      0.96      2728
          1       0.90      0.10      0.18       272

avg / total       0.92      0.92      0.89      3000

[[2725    3]
 [ 245   27]]
LR, N-Gram Vectors:  0.9173333333333333


In [19]:
# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print( "LR, CharLevel Vectors: ", accuracy)

             precision    recall  f1-score   support

          0       0.93      1.00      0.96      2728
          1       0.93      0.21      0.34       272

avg / total       0.93      0.93      0.90      3000

[[2724    4]
 [ 216   56]]
LR, CharLevel Vectors:  0.9266666666666666


In [21]:
# 3.3 Implementing a SVM Model
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_count, train_y, xvalid_count)
print("SVM, Count Vectors: ", accuracy)

             precision    recall  f1-score   support

          0       0.91      1.00      0.95      2728
          1       0.00      0.00      0.00       272

avg / total       0.83      0.91      0.87      3000

[[2728    0]
 [ 272    0]]
SVM, Count Vectors:  0.9093333333333333


In [22]:
accuracy = train_model(svm.SVC(),  xtrain_tfidf, train_y, xvalid_tfidf)
print("SVM, Word-Level TF-IDF: ", accuracy)

             precision    recall  f1-score   support

          0       0.91      1.00      0.95      2728
          1       0.00      0.00      0.00       272

avg / total       0.83      0.91      0.87      3000

[[2728    0]
 [ 272    0]]
SVM, Word-Level TF-IDF:  0.9093333333333333


In [27]:
# 3.4 Bagging Model
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print( "RF, Count Vectors: ", accuracy)

             precision    recall  f1-score   support

          0       0.96      0.99      0.97      2728
          1       0.86      0.57      0.69       272

avg / total       0.95      0.95      0.95      3000

[[2703   25]
 [ 117  155]]
RF, Count Vectors:  0.9526666666666667


In [28]:

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("RF, WordLevel TF-IDF: ", accuracy)

             precision    recall  f1-score   support

          0       0.93      0.99      0.96      2728
          1       0.67      0.28      0.39       272

avg / total       0.91      0.92      0.91      3000

[[2690   38]
 [ 196   76]]
RF, WordLevel TF-IDF:  0.922


In [29]:
# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print("RF, N-Gram Vectors: ", accuracy)

             precision    recall  f1-score   support

          0       0.96      0.99      0.97      2728
          1       0.83      0.59      0.69       272

avg / total       0.95      0.95      0.95      3000

[[2695   33]
 [ 111  161]]
RF, N-Gram Vectors:  0.952


In [30]:
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("RF, CharLevel Vectors: ", accuracy)

             precision    recall  f1-score   support

          0       0.93      1.00      0.96      2728
          1       0.92      0.29      0.45       272

avg / total       0.93      0.93      0.92      3000

[[2721    7]
 [ 192   80]]
RF, CharLevel Vectors:  0.9336666666666666


In [31]:
### 3.5 Boosting Model

# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print( "Xgb, Count Vectors: ", accuracy)

             precision    recall  f1-score   support

          0       0.95      0.99      0.97      2728
          1       0.90      0.49      0.64       272

avg / total       0.95      0.95      0.94      3000

[[2713   15]
 [ 138  134]]
Xgb, Count Vectors:  0.949


C:\Users\Yutong\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [32]:
# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print("Xgb, WordLevel TF-IDF: ", accuracy)

             precision    recall  f1-score   support

          0       0.95      0.99      0.97      2728
          1       0.89      0.49      0.63       272

avg / total       0.95      0.95      0.94      3000

[[2711   17]
 [ 138  134]]
Xgb, WordLevel TF-IDF:  0.9483333333333334


C:\Users\Yutong\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [33]:
# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("Xgb, N-Gram Vectors: ", accuracy)

             precision    recall  f1-score   support

          0       0.96      0.99      0.97      2728
          1       0.84      0.53      0.65       272

avg / total       0.94      0.95      0.94      3000

[[2701   27]
 [ 127  145]]
Xgb, N-Gram Vectors:  0.9486666666666667


C:\Users\Yutong\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [34]:
# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print("Xgb, CharLevel Vectors: ", accuracy)

             precision    recall  f1-score   support

          0       0.96      0.99      0.97      2728
          1       0.84      0.53      0.65       272

avg / total       0.94      0.95      0.94      3000

[[2701   27]
 [ 127  145]]
Xgb, CharLevel Vectors:  0.9486666666666667


C:\Users\Yutong\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [35]:
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="tanh")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

classifier = create_cnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print( "CNN, Word Embeddings",  accuracy)



Epoch 1/1
7000/7000 [==============================] - 11s 2ms/step - loss: 0.3100 - accuracy: 0.9001 1s - l
             precision    recall  f1-score   support

          0       0.92      1.00      0.96      2728
          1       0.96      0.17      0.28       272

avg / total       0.93      0.92      0.90      3000

[[2726    2]
 [ 227   45]]
CNN, Word Embeddings 0.9236666666666666


In [36]:
# 3.7.2 Recurrent Neural Network – LSTM
def create_rnn_lstm():
     # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="tanh")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_lstm()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print( "RNN-LSTM, Word Embeddings",  accuracy)

Epoch 1/1
7000/7000 [==============================] - 74s 11ms/step - loss: 0.3038
             precision    recall  f1-score   support

          0       0.92      1.00      0.96      2728
          1       1.00      0.10      0.17       272

avg / total       0.92      0.92      0.89      3000

[[2728    0]
 [ 246   26]]
RNN-LSTM, Word Embeddings 0.918


In [37]:
# 3.7.3 Recurrent Neural Network – GRU
def create_rnn_gru():
     # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the GRU Layer
    lstm_layer = layers.GRU(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="tanh")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_gru()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)

Epoch 1/1
7000/7000 [==============================] - 113s 16ms/step - loss: 0.3113
             precision    recall  f1-score   support

          0       0.93      0.99      0.96      2728
          1       0.73      0.22      0.34       272

avg / total       0.91      0.92      0.90      3000

[[2706   22]
 [ 211   61]]


In [38]:
# 3.7.4 Bidirectional RNN
def create_bidirectional_rnn():
      # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_bidirectional_rnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print( "RNN-Bidirectional, Word Embeddings",  accuracy)

Epoch 1/1
7000/7000 [==============================] - 119s 17ms/step - loss: 0.3234
             precision    recall  f1-score   support

          0       0.92      1.00      0.96      2728
          1       1.00      0.07      0.13       272

avg / total       0.92      0.92      0.88      3000

[[2728    0]
 [ 253   19]]
RNN-Bidirectional, Word Embeddings 0.9156666666666666
